In [1]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import ElasticNetCV

In [2]:
d = pd.read_csv('venue_db.csv')
pn = {1: 'good', 0: 'bad'}
print(f'total rows: {len(d)}')

c = Counter(d['is_ok'])
print({pn[_]: c[_] for _ in c})

total rows: 2884
{'good': 2721, 'bad': 163}


In [3]:
pos_ = d[d['is_ok'] == 1]
neg_ = d[d['is_ok'] == 0]

while len(neg_) < len(pos_):
    neg_ = pd.concat([neg_, neg_.sample(frac=0.2)])
    
d = pd.concat([pos_, neg_]).sample(frac=1.)

In [4]:
X = d['venue']
y = d['is_ok']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)
print(f'training set: {len(X_train)}, testing set: {len(X_test)}')

training set: 4014, testing set: 1721


In [5]:
# this will produce a feature union with components called like 'countvectorizer'
features = make_union(CountVectorizer(strip_accents='ascii', 
                                      lowercase=True, 
                                      stop_words='english', 
                                      ngram_range=(1, 3), 
                                      analyzer='word'))

In [6]:
cls = {'randomforestclassifier': RandomForestClassifier(),
       'svc': SVC(),
       'elasticnetcv': ElasticNetCV(max_iter=100)}

ps = {'randomforestclassifier': {'n_estimators': (100,200,300)},
        'svc': {'class_weight': ('balanced', None),
                'C': (1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1)},
         'elasticnetcv': {'l1_ratio': (.1, .5, .7, .9, .95, .99, 1)}}


for cl in cls:
    
    print(f'running {cl.upper()}...')
    
    pipeline = make_pipeline(features, cls[cl])
    
    pars = {f'{cl}__{p}': ps[cl][p] for p in ps[cl]}
    
    grid_search = GridSearchCV(pipeline, param_grid=pars)
    
    grid_search.fit(X_train, y_train.values.ravel())
    
    yh = grid_search.predict(X_test)
    
    print(classification_report(y_test, yh))
    
    print('confusion matrix')
    
    print(confusion_matrix(y_test, yh))
    
    print('incorrectly labelled as BAD')
    
    print(X_test[list(yh - y_test < 0)])
    
    print('incorrectly labelled as GOOD')
    
    print(X_test[list(yh - y_test > 0)])

running RANDOMFORESTCLASSIFIER...
             precision    recall  f1-score   support

          0       0.98      1.00      0.99       904
          1       1.00      0.98      0.99       817

avg / total       0.99      0.99      0.99      1721

confusion matrix
[[903   1]
 [ 20 797]]
incorrectly labelled as BAD
1796                            gold coast events centre
1540                   rhythmbeat cruises sydney harbour
866                         also shed dance party events
1171                              peninsula central pier
546                              anz stadium memberships
322                          bus bay charles west avenue
1686                                    gold coast buses
1438                                     qld misc events
709                       depart pier 26 darling harbour
1519            brisbane transport chermside interchange
329                             chevron festival gardens
1467                      melbourne park car park events

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/pyth

ValueError: Mix type of y not allowed, got types {'binary', 'continuous'}